In [1]:
import os
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import pandas as pd

def geography(left, right, bottom, top):
    #
    resolution = 12
    #
    upperindex = (90-top)*resolution
    lowerindex = (90-bottom)*resolution
    leftindex = (180+left)*resolution
    rightindex = (180+right)*resolution
    #
    rgnshape = (lowerindex-upperindex, rightindex-leftindex)
    #
    img_extent = (left, right, bottom, top)
    
    return upperindex, lowerindex, leftindex, rightindex, rgnshape, img_extent

In [ ]:
# this calculation should be resumed 
# after executing wsi/pre/updown/updwon_availability

city_len = 1860
ex_flg = True

h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
pop_path = f'{h08dir}/global_city/dat/pop_tot_/GPW4ag__20100000.gl5'
pop = np.fromfile(pop_path, dtype='float32').reshape(2160, 4320)
riv_path = f'{h08dir}/wsi/dat/riv_out_/W5E5LR__00000000.gl5'
rivout = np.fromfile(riv_path, dtype='float32').reshape(2160, 4320)
msk_dir = f'{h08dir}/global_city/dat/vld_cty_'
prf_dir = f'{h08dir}/global_city/dat/cty_prf_'

updown_path = f'{h08dir}/wsi/dat/availablewater/updown_availability_100km_abandon_revised.txt'
df_new = pd.read_csv(updown_path, delimiter='|', header=None)
df_new.columns = df_new.iloc[0]
df_new = df_new[1:].reset_index(drop=True)
df_new = df_new.infer_objects()

# intake
distance = 100

# lonlat
lonlat_path = f'{h08dir}/global_city/dat/cty_lst_/gpw4/WUP2018_300k_2010.txt'
with open(lonlat_path, "r") as input_file:
    lines = input_file.readlines()

CMY = []
world = np.zeros((2160, 4320))

for i in range(city_len):
    city_num = i+1
    msk_path = f'{msk_dir}/city_{city_num:08}.gl5'
    prf_path = f'{prf_dir}/vld_cty_/city_{city_num:08}.gl5'
    
    if not os.path.exists(prf_path):
        print(f'{i+1} is invalid prf')
        
    else:
        # population
        msk = np.fromfile(msk_path, dtype='float32').reshape(2160, 4320)
        msk_int = msk.astype(int)
        
        # lonlat
        line = lines[i]
        parts = line.split('\t')
        parts = [item.strip() for item in parts]
        cnt_lat = float(parts[1])
        cnt_lon = float(parts[2])
        city_name = parts[4].replace("\"", "").replace("?", "").replace("/", "")
        print('lonlat', city_num, city_name, cnt_lon, cnt_lat)

        # cmy = cubic meter(m3) per capita(cap-1) per year(year-1)
        
        cmy = df_new.loc[df_new['index'] == str(city_num), 'mpcy_int'].values[0]
        CMY.append([city_num, cnt_lon, cnt_lat, cmy])

        # if world already has some value else of 0 at msk == 1, it's overlap and not added
        if np.sum(world[msk_int == 1]) < 1:
            world[msk_int == 1] = cmy
        else:
            print('overlap')
        print('cmy', city_num, cmy)
        print('--------------------------------------------')
        
upperindex, lowerindex, leftindex, rightindex, rgnshape, img_extent = geography(135, 145, 30, 40)
region = world[upperindex:lowerindex, leftindex:rightindex]
region_masked = np.ma.masked_where(region==0, region)
plt.imshow(region_masked)
plt.show()
    
if ex_flg is True:
    # save 1860 array
    cmy_array = np.array(CMY)
    savepath = f'{h08dir}/wsi/dat/cmy/cmy_citylonlat_{distance}km_abandon_revised.npy'
    np.save(savepath, cmy_array)
    print(f'{savepath} is saved')
    
    savepath = f'{h08dir}/wsi/dat/cmy/world_citygrid_{distance}km_abandon_revised.npy'
    np.save(savepath, world)
    print(f'{savepath} is saved')

lonlat 1 Tokyo 139.69171 35.6895
cmy 1 547.0431026426597
--------------------------------------------
lonlat 2 Delhi 77.21667 28.66667
cmy 2 818.1604314423055
--------------------------------------------
lonlat 3 Shanghai 121.45806 31.22222
cmy 3 865.1976258489309
--------------------------------------------
lonlat 4 Ciudad de México (Mexico City) -99.141869 19.427318
cmy 4 120.33867764606957
--------------------------------------------
lonlat 5 São Paulo -46.63611 -23.5475
cmy 5 212.66356604651162
--------------------------------------------
lonlat 6 Kinki M.M.A. (Osaka) 135.553823 34.675834
cmy 6 1603.4898075190176
--------------------------------------------
lonlat 7 New York-Newark -74.003663 40.717042
cmy 7 2847.5196171406665
--------------------------------------------
lonlat 8 Mumbai (Bombay) 72.880838 19.073975
cmy 8 1052.9687438873375
--------------------------------------------
lonlat 9 Al-Qahirah (Cairo) 31.239411 30.039173
cmy 9 41312.29777306079
---------------------------